# NFL Jersey Number Recognition to Track Players

# Recognise jersey numbers in new video frames

In [3]:
import os
import time
import random
import cv2 as cv
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.utils import class_weight

In [ ]:
test_helmet_df = pd.read_csv("nfl-health-and-safety-helmet-assignment/test_baseline_helmets.csv")
test_tracking_df = pd.read_csv("nfl-health-and-safety-helmet-assignment/test_player_tracking.csv")

In [2]:
from keras.models import load_model
model = load_model('best_model.h5')

In [10]:
def extract_player_jersey(video_frame_name, display=False):
    '''
    Get the helmet boxes for a frame and apply the model.
    If a player is predicted twice, keeps the prediction
    with the highest confidence score.
    
    NURC Lacrosse notes
    Code has been changed slightly from the NFL dataset sample code - removes
    the "find_team" aspect of the search, among some other NFL-specific tasks
    '''
    
    predictions = []
    img = np.array(Image.open(test_frames_folder+"/"+str(video_frame_name)))

    baseline_boxes = np.array([np.array([row.left, row.top, row.left+row.width, row.top+row.height ]) \
                               for idx, row in frame_df.iterrows()])
    for idx, box in enumerate(baseline_boxes):
        box_centre = int(box[0]+round((box[2]-box[0])/2))
        jersey_box = img[box[3]-24:box[3]+40,box_centre-32:box_centre+32,:]
        
        #seems each input image has to be 64x64 for the script to work.  can try to resize
        #the image to get it to fit that case
        if jersey_box.shape==(64,64,3):
            result = model.predict(np.array([np.array(jersey_box)]))
            predicted_jersey_number = np.argmax(result)
            confidence = result[0][np.argmax(result)] 
            confidence_threshold = 0.90
       
            if confidence>confidence_threshold:
                player_already_detected = [(i, item) for i, item in enumerate(predictions) if item["label"] == predicted_player_code]
                prediction_data = {"video_frame":video_frame_name.replace(".png",""), 
                                        "label":predicted_jersey_number,
                                        "left":frame_df.iloc[idx].left,
                                        "width":frame_df.iloc[idx].width,	
                                        "top":frame_df.iloc[idx].top,
                                        "height":frame_df.iloc[idx].height,
                                        "confidence":confidence}

                if player_already_detected==[]:
                    predictions.append(prediction_data)
                else:
                    if player_already_detected[0][1]['confidence']<confidence:
                        dict_index_to_remove = player_already_detected[0][0]
                        del predictions[dict_index_to_remove]
                        predictions.append(prediction_data)

                if display:
                    print(predicted_player_code, confidence)
                    plt.imshow(jersey_box)
                    plt.show()
        
    
    return predictions

For this run, we'll only apply prediction on the frames taken from the endzone as the jerseys are often easier to see.

In [11]:
test_frames_folder = "womens_lax_test_data1"
frame_list = os.listdir(test_frames_folder)
random.seed(42)
frames_to_test = random.sample(frame_list, 6) 

In [12]:
extract_player_jersey(frames_to_test[0], display=True)

NameError: name 'frame_df' is not defined

In [ ]:
extract_player_jersey(frames_to_test[1], display=True)

In [ ]:
extract_player_jersey(frames_to_test[2], display=True)

In [ ]:
extract_player_jersey(frames_to_test[3], display=True)

In [ ]:
extract_player_jersey(frames_to_test[4], display=True)

In [ ]:
extract_player_jersey(frames_to_test[5], display=True)

I have created the function to submit the predictions but it's still early days to be spamming the leaderboard yet!

In [ ]:
def predict_for_submission(frame_list):
    prediction_list = []
    with tqdm(total=len(frame_list)) as pbar:
        for video_frame in frame_list:
            prediction = extract_player_jersey(video_frame)
            prediction_list += prediction
            pbar.update(1)
    return pd.DataFrame(prediction_list)

In [ ]:
predict_for_submission(frame_list[:30])

## Thanks for reading this notebook! If you found this notebook helpful, please give it an upvote. It is always greatly appreciated!

In [ ]:
!rm -rf ../working/test_frames